In [1]:

#Libraries Required
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
#Upload the datasets
movie = pd.read_csv('/Users/ssahin/Desktop/Movie_Recommendation/movie_lens_dataset/movie.csv')
rating = pd.read_csv('/Users/ssahin/Desktop/Movie_Recommendation/movie_lens_dataset/rating.csv')
df = movie.merge(rating, how="left", on="movieId")
df.drop(columns=['timestamp'], inplace=True)
df.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5


In [2]:
############################ 10% Excluded  ############################
max_count = list(pd.DataFrame(df["title"].value_counts()).max())[0]
comment_counts = pd.DataFrame(df["title"].value_counts())
rare_movies = comment_counts[comment_counts["title"] <= max_count*0.1].index
common_movies = df[~df["title"].isin(rare_movies)]

In [3]:
movie_df = common_movies.pivot_table(index="title", columns="userId", values="rating")
movie_df.fillna(0, inplace=True)
movie_df.head()

userId,1.0,2.0,3.0,4.0,5.0,...,138489.0,138490.0,138491.0,138492.0,138493.0
title,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0
101 Dalmatians (One Hundred and One Dalmatians) (1961),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,0.0,3.5
12 Angry Men (1957),0.0,0.0,0.0,0.0,0.0,...,4.5,0.0,0.0,0.0,4.0
"13th Warrior, The (1999)",0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


In [4]:
######################################
# KNN Model
######################################
from scipy.sparse import csr_matrix
movie_matrix = csr_matrix(movie_df)
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=11, algorithm='brute', metric='cosine')
model.fit(movie_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=11)

In [5]:
movie_name = "Braveheart (1995)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))

In [6]:
recommend_list = suggestions.flatten().tolist()
print(recommend_list)

[109, 264, 372, 273, 665, 605, 531, 597, 177, 44, 699]


In [7]:
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
print(recommend_list_2)

['Braveheart (1995)', 'Forrest Gump (1994)', 'Jurassic Park (1993)', 'Fugitive, The (1993)', 'Terminator 2: Judgment Day (1991)', 'Silence of the Lambs, The (1991)', 'Pulp Fiction (1994)', 'Shawshank Redemption, The (1994)', 'Dances with Wolves (1990)', 'Apollo 13 (1995)', 'True Lies (1994)']


In [8]:
distance_list = distances.flatten().tolist()
print(distance_list)

[0.0, 0.3499663877885446, 0.3638208705446654, 0.3709768724456143, 0.37483416431432537, 0.39062417471598476, 0.3915920500610316, 0.395565137644744, 0.3964883099645561, 0.40340683655247145, 0.4127420214845938]


In [9]:
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df

,title,distances
0,Braveheart (1995),0.000000
1,Forrest Gump (1994),0.349966
2,Jurassic Park (1993),0.363821
3,"Fugitive, The (1993)",0.370977
4,Terminator 2: Judgment Day (1991),0.374834
5,"Silence of the Lambs, The (1991)",0.390624
6,Pulp Fiction (1994),0.391592
7,"Shawshank Redemption, The (1994)",0.395565
8,Dances with Wolves (1990),0.396488
9,Apollo 13 (1995),0.403407


In [10]:
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df.head()

,title,distances,genres,rating
0,Braveheart (1995),0.0,Action|Drama|War,4.0
1,Braveheart (1995),0.0,Action|Drama|War,4.0
2,Braveheart (1995),0.0,Action|Drama|War,5.0
3,Braveheart (1995),0.0,Action|Drama|War,4.0
4,Braveheart (1995),0.0,Action|Drama|War,4.0


In [11]:
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df

,title,distances,genres,rating,averageRating
0,Braveheart (1995),0.000000,Action|Drama|War,4.0,4.042534
1,Braveheart (1995),0.000000,Action|Drama|War,4.0,4.042534
2,Braveheart (1995),0.000000,Action|Drama|War,5.0,4.042534
3,Braveheart (1995),0.000000,Action|Drama|War,4.0,4.042534
4,Braveheart (1995),0.000000,Action|Drama|War,4.0,4.042534
...,...,...,...,...,...
610595,True Lies (1994),0.412742,Action|Adventure|Comedy|Romance|Thriller,4.0,3.491149
610596,True Lies (1994),0.412742,Action|Adventure|Comedy|Romance|Thriller,4.0,3.491149
610597,True Lies (1994),0.412742,Action|Adventure|Comedy|Romance|Thriller,1.0,3.491149
610598,True Lies (1994),0.412742,Action|Adventure|Comedy|Romance|Thriller,3.0,3.491149


In [12]:
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df

,title,distances,genres,averageRating
0,Braveheart (1995),0.000000,Action|Drama|War,4.042534
53769,Forrest Gump (1994),0.349966,Comedy|Drama|Romance|War,4.029000
119941,Jurassic Park (1993),0.363821,Action|Adventure|Sci-Fi|Thriller,3.664741
179656,"Fugitive, The (1993)",0.370977,Thriller,3.985690
229237,Terminator 2: Judgment Day (1991),0.374834,Action|Sci-Fi,3.931954
281481,"Silence of the Lambs, The (1991)",0.390624,Crime|Horror|Thriller,4.177057
344780,Pulp Fiction (1994),0.391592,Comedy|Crime|Drama|Thriller,4.174231
412090,"Shawshank Redemption, The (1994)",0.395565,Crime|Drama,4.446990
475456,Dances with Wolves (1990),0.396488,Adventure|Drama|Western,3.728465
519664,Apollo 13 (1995),0.403407,Adventure|Drama|IMAX,3.868598


In [13]:
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Braveheart (1995) Movies are recommended by KNN 


,title,averageRating,distances,genres
1,Forrest Gump (1994),4.029000,0.349966,Comedy|Drama|Romance|War
2,Jurassic Park (1993),3.664741,0.363821,Action|Adventure|Sci-Fi|Thriller
3,"Fugitive, The (1993)",3.985690,0.370977,Thriller
4,Terminator 2: Judgment Day (1991),3.931954,0.374834,Action|Sci-Fi
5,"Silence of the Lambs, The (1991)",4.177057,0.390624,Crime|Horror|Thriller
6,Pulp Fiction (1994),4.174231,0.391592,Comedy|Crime|Drama|Thriller
7,"Shawshank Redemption, The (1994)",4.446990,0.395565,Crime|Drama
8,Dances with Wolves (1990),3.728465,0.396488,Adventure|Drama|Western
9,Apollo 13 (1995),3.868598,0.403407,Adventure|Drama|IMAX
10,True Lies (1994),3.491149,0.412742,Action|Adventure|Comedy|Romance|Thriller


In [14]:
movie_name = "Braveheart (1995)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))
recommend_list = suggestions.flatten().tolist()
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
distance_list = distances.flatten().tolist()
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Braveheart (1995) Movies are recommended by KNN 


,title,averageRating,distances,genres
1,Forrest Gump (1994),4.029000,0.349966,Comedy|Drama|Romance|War
2,Jurassic Park (1993),3.664741,0.363821,Action|Adventure|Sci-Fi|Thriller
3,"Fugitive, The (1993)",3.985690,0.370977,Thriller
4,Terminator 2: Judgment Day (1991),3.931954,0.374834,Action|Sci-Fi
5,"Silence of the Lambs, The (1991)",4.177057,0.390624,Crime|Horror|Thriller
6,Pulp Fiction (1994),4.174231,0.391592,Comedy|Crime|Drama|Thriller
7,"Shawshank Redemption, The (1994)",4.446990,0.395565,Crime|Drama
8,Dances with Wolves (1990),3.728465,0.396488,Adventure|Drama|Western
9,Apollo 13 (1995),3.868598,0.403407,Adventure|Drama|IMAX
10,True Lies (1994),3.491149,0.412742,Action|Adventure|Comedy|Romance|Thriller


In [15]:
movie_name = "Schindler's List (1993)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))
recommend_list = suggestions.flatten().tolist()
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
distance_list = distances.flatten().tolist()
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN: '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Schindler's List (1993) Movies are recommended by KNN: 


,title,averageRating,distances,genres
1,"Shawshank Redemption, The (1994)",4.446990,0.377850,Crime|Drama
2,"Silence of the Lambs, The (1991)",4.177057,0.388535,Crime|Horror|Thriller
3,Forrest Gump (1994),4.029000,0.406595,Comedy|Drama|Romance|War
4,Pulp Fiction (1994),4.174231,0.412246,Comedy|Crime|Drama|Thriller
5,Braveheart (1995),4.042534,0.450639,Action|Drama|War
6,"Usual Suspects, The (1995)",4.334372,0.453794,Crime|Mystery|Thriller
7,Jurassic Park (1993),3.664741,0.456632,Action|Adventure|Sci-Fi|Thriller
8,"Fugitive, The (1993)",3.985690,0.471066,Thriller
9,Terminator 2: Judgment Day (1991),3.931954,0.475619,Action|Sci-Fi
10,Apollo 13 (1995),3.868598,0.480919,Adventure|Drama|IMAX


In [16]:
movie_name = "Fast and the Furious, The (2001)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))
recommend_list = suggestions.flatten().tolist()
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
distance_list = distances.flatten().tolist()
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Fast and the Furious, The (2001) Movies are recommended by KNN 


,title,averageRating,distances,genres
1,Gone in 60 Seconds (2000),3.066489,0.564625,Action|Crime
2,Lara Croft: Tomb Raider (2001),2.758028,0.596207,Action|Adventure
3,"Mummy Returns, The (2001)",2.937224,0.610280,Action|Adventure|Comedy|Thriller
4,American Pie 2 (2001),2.988147,0.618905,Comedy
5,Men in Black II (a.k.a. MIIB) (a.k.a. MIB 2) (...,2.922229,0.628394,Action|Comedy|Sci-Fi
6,Pearl Harbor (2001),2.820810,0.633592,Action|Drama|Romance|War
7,Spider-Man (2002),3.569176,0.650616,Action|Adventure|Sci-Fi|Thriller
8,"Italian Job, The (2003)",3.573480,0.651042,Action|Crime
9,Ocean's Eleven (2001),3.791851,0.651267,Crime|Thriller
10,Bruce Almighty (2003),3.214816,0.651626,Comedy|Drama|Fantasy|Romance


In [17]:
movie_name = "Matrix Revolutions, The (2003)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))
recommend_list = suggestions.flatten().tolist()
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
distance_list = distances.flatten().tolist()
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Matrix Revolutions, The (2003) Movies are recommended by KNN 


,title,averageRating,distances,genres
1,"Matrix Reloaded, The (2003)",3.296389,0.265136,Action|Adventure|Sci-Fi|Thriller|IMAX
2,"I, Robot (2004)",3.459808,0.473079,Action|Adventure|Sci-Fi|Thriller
3,Star Wars: Episode III - Revenge of the Sith (...,3.465821,0.492831,Action|Adventure|Sci-Fi
4,Star Wars: Episode II - Attack of the Clones (...,3.108584,0.495090,Action|Adventure|Sci-Fi|IMAX
5,Terminator 3: Rise of the Machines (2003),3.113221,0.497518,Action|Adventure|Sci-Fi
6,X2: X-Men United (2003),3.681243,0.505356,Action|Adventure|Sci-Fi|Thriller
7,Spider-Man 2 (2004),3.551107,0.509700,Action|Adventure|Sci-Fi|IMAX
8,Kill Bill: Vol. 1 (2003),3.901338,0.515894,Action|Crime|Thriller
9,Spider-Man (2002),3.569176,0.516445,Action|Adventure|Sci-Fi|Thriller
10,"Lord of the Rings: The Return of the King, The...",4.142382,0.517477,Action|Adventure|Drama|Fantasy


In [18]:
movie_name = "Enemy at the Gates (2001)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))
recommend_list = suggestions.flatten().tolist()
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
distance_list = distances.flatten().tolist()
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Enemy at the Gates (2001) Movies are recommended by KNN 


,title,averageRating,distances,genres
1,Black Hawk Down (2001),3.686862,0.544360,Action|Drama|War
2,Training Day (2001),3.677569,0.609618,Crime|Drama|Thriller
3,Gladiator (2000),3.952248,0.616946,Action|Adventure|Drama
4,"Last Samurai, The (2003)",3.694285,0.622357,Action|Adventure|Drama|War
5,Road to Perdition (2002),3.685840,0.624631,Crime|Drama
6,"Bourne Identity, The (2002)",3.901050,0.624982,Action|Mystery|Thriller
7,Ocean's Eleven (2001),3.791851,0.635757,Crime|Thriller
8,Cast Away (2000),3.616454,0.645997,Drama
9,Snatch (2000),4.042147,0.647059,Comedy|Crime|Thriller
10,Minority Report (2002),3.664897,0.648157,Action|Crime|Mystery|Sci-Fi|Thriller


In [19]:
movie_name = "Pulp Fiction (1994)"
index_no = list(movie_df.index).index(movie_name)
distances, suggestions = model.kneighbors(movie_df.iloc[index_no, :].values.reshape(1, -1))
recommend_list = suggestions.flatten().tolist()
recommend_list_2 = []
for i in range(len(recommend_list)):
    recommend_list_2.append(movie_df.index[recommend_list[i]])
distance_list = distances.flatten().tolist()
recommend_df = pd.DataFrame({'title':recommend_list_2,'distances':distance_list})
recommend_df = recommend_df.merge(df,how='left',on='title')
recommend_df.drop(columns = ["userId","movieId"],inplace = True)
recommend_df['averageRating'] = recommend_df.groupby('title')['rating'].transform('mean')
recommend_df.drop_duplicates(subset = "title",inplace = True)
recommend_df.drop(columns = ["rating"],inplace = True)
recommend_df.sort_values(by="distances",ascending=True,inplace=True)
recommend_df.reset_index(drop=True,inplace=True)
recommend_df = recommend_df[["title","averageRating","distances","genres"]]
print('{0} Movies are recommended by KNN '.format(recommend_list_2[0]))
recommend_df.iloc[1:]

Pulp Fiction (1994) Movies are recommended by KNN 


,title,averageRating,distances,genres
1,"Silence of the Lambs, The (1991)",4.177057,0.285908,Crime|Horror|Thriller
2,"Shawshank Redemption, The (1994)",4.446990,0.318771,Crime|Drama
3,Seven (a.k.a. Se7en) (1995),4.053493,0.337351,Mystery|Thriller
4,"Usual Suspects, The (1995)",4.334372,0.343228,Crime|Mystery|Thriller
5,Forrest Gump (1994),4.029000,0.344074,Comedy|Drama|Romance|War
6,Jurassic Park (1993),3.664741,0.390726,Action|Adventure|Sci-Fi|Thriller
7,Terminator 2: Judgment Day (1991),3.931954,0.390794,Action|Sci-Fi
8,Braveheart (1995),4.042534,0.391592,Action|Drama|War
9,Batman (1989),3.402365,0.392507,Action|Crime|Thriller
10,"Fugitive, The (1993)",3.985690,0.400923,Thriller
